### Setup

In [ ]:
import wandb
from lib.utils.modelUtils import ModelUtils
from lib.utils.dataUtils import DataUtils
from lib.utils.sweep_configs import sweep_config
from lib.models.CNN import cnns
import random
import numpy as np
import keras.backend as K
import tensorflow as tf
from os import environ

# Set random seeds
environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(hash('setting random seeds') % 2 ** 32 - 1)
np.random.seed(hash('improves reproducibility') % 2 ** 32 - 1)
tf.random.set_seed(hash('so that runs are repeatable'))

print("# GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if len(tf.config.list_physical_devices('GPU')):
    print('GPU Device Name', tf.test.gpu_device_name())

LABELS = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

### Data

In [ ]:
data_utils = DataUtils('../data', '../outputs')
train_data, val_data = data_utils.load_data(train = True)

### Model

In [ ]:
K.clear_session()
model = ModelUtils(
    model = cnns['simple-net'],
    training_data = train_data,
    validation_data = val_data,
    batch_size = 512
)

### Train

In [ ]:
wandb.login()
sweep_id = wandb.sweep(sweep_config, entity='manelfideles', project="intel-icc")
wandb.agent(
    sweep_id, 
    model.train,
    count = 5
)
wandb.finish()

### Results

We're facing extreme overfitting. Validation results are several times worse than training results.
- Use data augmentation: One way to combat overfitting is to use data augmentation. By creating variations of the training data, such as flipping or rotating the images, you can increase the size of the training set and reduce overfitting. In TensorFlow, you can use the ImageDataGenerator class to apply data augmentation.
- Add regularization: Regularization is a technique that adds a penalty term to the loss function to discourage overfitting. In TensorFlow, you can add regularization to the convolutional and dense layers using the kernel_regularizer argument. For example, you can use L1 or L2 regularization, or a combination of both.
- Reduce the model's complexity: If the model is too complex for the size of the dataset, it can overfit. One way to reduce the complexity of the model is to reduce the number of convolutional or dense layers, or to decrease the number of filters or neurons in each layer.
- Increase the dropout rate: Dropout is a technique that randomly drops out some neurons during training to prevent overfitting. If the model is overfitting, you can increase the dropout rate to drop more neurons.
- Use early stopping: Early stopping is a technique that stops the training process when the validation loss stops improving. This prevents the model from overfitting to the training data by stopping the training before the model starts to overfit. In TensorFlow, you can use the EarlyStopping callback to implement early stopping.